<a href="https://colab.research.google.com/github/ReutFarkash/useful/blob/main/TensorFlow_Tutorial_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[TensorFlow Tutorial 15 - Customizing Model.Fit](https://www.youtube.com/watch?v=S6tLSI8bjGs&ab_channel=AladdinPersson)

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

In [3]:
model = keras.Sequential(
    [
     layers.Input(shape=(28, 28, 1)),
     layers.Conv2D(64, 3, padding='same'),
     layers.ReLU(),
     layers.Conv2D(128, 3, padding='same'),
     layers.ReLU(),
     layers.Flatten(),
     layers.Dense(10),
    ],
    name='model',
)

In [4]:
class CustomFit(keras.Model):
  def __init__(self, model):
    super(CustomFit, self).__init__()
    self.model = model
  
  def train_step(self, data):
    x, y = data

    with tf.GradientTape() as tape:
      y_pred = self.model(x, training=True)
      loss = self.compiled_loss(y, y_pred)
    
    training_vars = self.trainable_variables
    gradients = tape.gradient(loss, training_vars)

    self.optimizer.apply_gradients(zip(gradients, training_vars))
    self.compiled_metrics.update_state(y, y_pred)

    return {m.name : m.result() for m in self.metrics}

In [5]:
training = CustomFit(model)
training.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
training.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1100 - accuracy: 0.9667
Epoch 2/2
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0409 - accuracy: 0.9872


In [6]:
class CustomFit(keras.Model):
  def __init__(self, model):
    super(CustomFit, self).__init__()
    self.model = model

  def compile(self, optimizer, loss):
    super(CustomFit, self).compile()
    self.optimizer = optimizer
    self.loss = loss

  def train_step(self, data):
    x, y = data

    with tf.GradientTape() as tape:
      y_pred = self.model(x, training=True)
      loss = self.loss(y, y_pred)
    
    training_vars = self.trainable_variables
    gradients = tape.gradient(loss, training_vars)

    self.optimizer.apply_gradients(zip(gradients, training_vars))
    acc_metric.update_state(y, y_pred)

    return {"loss" : loss, "accuracy" : acc_metric.result()}
  
  def test_step(self, data):
    x, y = data

    y_pred = self.model(x, training=False)
    loss = self.loss(y, y_pred)
    acc_metric.update_state(y, y_pred)

    return {"loss": loss, "accuracy": acc_metric.result()}

acc_metric = keras.metrics.SparseCategoricalAccuracy(name="accuracy")

training = CustomFit(model)
training.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)
training.fit(x_train, y_train, batch_size=64, epochs=2)
training.evaluate(x_test, y_test, batch_size=32)

Epoch 1/2
938/938 [==============================] - 8s 8ms/step - loss: 0.0114 - accuracy: 0.9967
Epoch 2/2
313/313 [==============================] - 1s 2ms/step - loss: 0.0380 - accuracy: 0.9972


[]